In [12]:
import json
from datetime import date
from pathlib import Path
from llama_index.core import StorageContext, VectorStoreIndex, Settings, Document
from llama_index.vector_stores.chroma.base import ChromaVectorStore
from llama_index.llms.groq import Groq
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
import chromadb
from dotenv import load_dotenv
import random
from faker import Faker
from llama_index.core import load_index_from_storage

# Initialize Faker for fake data generation
fake = Faker()

# Load environment variables
load_dotenv()

# =============================================
# 1. Generate Company Data
# =============================================

def generate_company():
    """Generate realistic company financial data"""
    return {
        "company": fake.company(),
        "symbol": fake.lexify(text="????").upper(),
        "cik": str(random.randint(10**9, 10**10)),
        "fiscal_year": random.randint(2018, 2023),
        "revenue": round(random.uniform(1e9, 500e9), 2),
        "net_income": round(random.uniform(1e8, 50e9), 2),
        "eps": round(random.uniform(1.0, 20.0), 2),
        "employees": random.randint(1000, 500000),
        "risk_factors": [fake.sentence() for _ in range(20)],
        "mdna": [fake.paragraph() for _ in range(10)]  # Management Discussion
    }

# Generate 50 companies (30+ pages of data)
companies = [generate_company() for _ in range(50)]

# =============================================
# 2. Fix JSON Serialization Error (Date Handling)
# =============================================

from datetime import datetime, date
import json

class CustomJSONEncoder(json.JSONEncoder):
    """Handles both date and datetime serialization"""
    def default(self, obj):
        if isinstance(obj, (date, datetime)):
            return obj.isoformat()
        return super().default(obj)

# =============================================
# 3. Create Persistent Vector Store
# =============================================

# Initialize ChromaDB client
chroma_client = chromadb.PersistentClient(path="./chroma_db")

# Create collection
chroma_collection = chroma_client.get_or_create_collection("sec_filings")

# Define vector store
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

# Create storage context
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# =============================================
# 4. Modified Document Creation with Proper Serialization
# =============================================

def create_sec_document(company):
    """Convert company data to structured SEC filing document"""
    sections = {
        "Business Overview": fake.paragraphs(nb=5),
        "Risk Factors": company["risk_factors"],
        "Management Discussion": company["mdna"],
        "Financial Statements": [
            f"Revenue: ${company['revenue']/1e9:.2f}B",
            f"Net Income: ${company['net_income']/1e9:.2f}B",
            f"EPS: ${company['eps']}",
            f"Employees: {company['employees']:,}"
        ]
    }
    
    full_text = "\n\n".join(
        [f"## {section}\n" + "\n".join(content)
         for section, content in sections.items()]
    )
    
    # Convert date to string for serialization
    filed_date = fake.date_between(start_date="-5y", end_date="today")
    
    return Document(
        text=full_text,
        metadata={
            "company": company["company"],
            "symbol": company["symbol"],
            "filing_type": random.choice(["10-K", "10-Q", "8-K"]),
            "fiscal_year": company["fiscal_year"],
            "filed_date": filed_date.isoformat(),
            "full_text": full_text  # Critical addition
        }
    )


# =============================================
# 5. Configure Settings with Persistence
# =============================================

# Initialize components
Settings.llm = Groq(model="mixtral-8x7b-32768")
Settings.embed_model = HuggingFaceEmbedding("BAAI/bge-large-en-v1.5")
Settings.chunk_size = 1024
Settings.chunk_overlap = 200

# =============================================
# 6. FIXED Index Creation and Loading
# =============================================

persist_dir = "./storage"
if not Path(persist_dir).exists():
    print("Creating new index...")
    sec_documents = [create_sec_document(c) for c in companies]
    
    # Create index with explicit docstore
    index = VectorStoreIndex.from_documents(
        sec_documents,
        storage_context=storage_context,
        show_progress=True,
        store_nodes_override=True  # Ensure nodes are stored
    )
    
    # Verify node storage
    if not index.docstore.docs:
        raise ValueError("Failed to store nodes in docstore!")
    
    # Persist properly
    index.storage_context.persist(persist_dir=persist_dir)
    
    # Get nodes directly from docstore
    all_nodes = list(index.docstore.docs.values())
    print(f"Created new index with {len(all_nodes)} nodes")
    print(f"Sample node text: {all_nodes[0].text[:200]}...")
else:
    print("Loading existing index...")
    storage_context = StorageContext.from_defaults(
        persist_dir=persist_dir,
        vector_store=vector_store
    )
    
    # Load with node verification
    index = load_index_from_storage(storage_context)
    
    # Get nodes with text recovery
    all_nodes = []
    for node_id in storage_context.docstore.docs:
        node = storage_context.docstore.get_node(node_id)
        
        # Ensure text exists
        if not hasattr(node, "text"):
            if hasattr(node, "content"):
                node.text = node.content
            elif "full_text" in node.metadata:
                node.text = node.metadata["full_text"]
            else:
                print(f"Skipping node {node_id} - no text found")
                continue
                
        all_nodes.append(node)
    
    print(f"Loaded {len(all_nodes)} nodes")
    if all_nodes:
        print(f"Sample node text: {all_nodes[0].text[:200]}...")

# Final validation
if not all_nodes:
    raise ValueError("""
    CRITICAL: No nodes loaded!
    Possible fixes:
    1. Delete ./storage and ./chroma_db folders
    2. Ensure create_sec_document() returns valid Documents
    3. Verify storage permissions
    """)

Creating new index...


Generating embeddings: 100%|██████████| 50/50 [01:08<00:00,  1.38s/it]


Created new index with 50 nodes
Sample node text: ## Business Overview
Itself society too meet. Community dream bar Congress blood expect.
Assume able food shoulder although at. Operation design reduce middle them far. Affect education accept south.
...


In [13]:
from llama_index.retrievers.bm25 import BM25Retriever
from llama_index.core import QueryBundle
from llama_index.core.schema import TextNode
import Stemmer  # PyStemmer library

# Convert documents to TextNodes with validation
text_nodes = []
for node in all_nodes:
    # Handle different node types and legacy formats
    if hasattr(node, "text"):
        clean_text = node.text.strip()
    elif hasattr(node, "content"):  # Backwards compatibility
        clean_text = node.content.strip()
    else:
        continue
    
    if not clean_text:
        continue
    
    text_nodes.append(TextNode(
        text=clean_text,
        metadata=node.metadata,
        excluded_embed_metadata_keys=["filed_date"],
        excluded_llm_metadata_keys=["filed_date"],
        id_=node.node_id
    ))

if not text_nodes:
    # Enhanced error diagnostics
    sample_nodes = [n.__dict__ for n in all_nodes[:3]]
    raise ValueError(f"""
    No valid text nodes found for BM25. 
    First 3 nodes sample: {json.dumps(sample_nodes, indent=2)}
    """)

# Initialize BM25 with explicit parameters
bm25_retriever = BM25Retriever.from_defaults(
    nodes=text_nodes,  # Use validated nodes
    similarity_top_k=10,
    stemmer=Stemmer.Stemmer("english"),
    token_pattern=r"(?u)\b\w+\b",  # Broader token pattern
    skip_stemming=False
)

vector_retriever = index.as_retriever(similarity_top_k=5)

def hybrid_search(query):
    """Production-grade hybrid search with score fusion"""
    query_bundle = QueryBundle(query_str=query)
    
    # Retrieve from both systems
    vector_results = vector_retriever.retrieve(query_bundle)
    bm25_results = bm25_retriever.retrieve(query_bundle)

    # RRF fusion with position weighting
    combined_scores = {}
    for rank, result in enumerate(vector_results):
        combined_scores[result.node.node_id] = combined_scores.get(result.node.node_id, 0) + 1/(rank + 60)
    
    for rank, result in enumerate(bm25_results):
        combined_scores[result.node.node_id] = combined_scores.get(result.node.node_id, 0) + 1/(rank + 60)

    # Sort and select top 7
    sorted_results = sorted(combined_scores.items(), 
                         key=lambda x: x[1], 
                         reverse=True)[:7]
    
    # Preserve order from both result sets
    final_results = []
    seen_ids = set()
    for result in vector_results + bm25_results:
        if result.node.node_id in dict(sorted_results) and result.node.node_id not in seen_ids:
            final_results.append(result)
            seen_ids.add(result.node.node_id)
    
    return final_results[:7]

In [14]:
# Example 1: Sector Analysis
response = hybrid_search(
    "Compare risk factors in the technology sector versus healthcare sector "
    "from 2020-2023 10-K filings. Identify common themes and sector-specific risks."
)

# Example 2: Financial Health Check
response = hybrid_search(
    "List companies with negative EPS growth but increasing R&D expenditure "
    "from their latest 10-Q filings. Include financial metrics and CEO commentary."
)

# Example 3: M&A Analysis
response = hybrid_search(
    "Identify all 8-K filings related to mergers and acquisitions in the "
    "past 3 years. Analyze deal sizes and strategic rationales provided."
)

In [15]:
import pandas as pd
import time
from datetime import datetime

class QueryMonitor:
    def __init__(self):
        # Initialize DataFrame with explicit column types
        self.queries = pd.DataFrame(columns=["timestamp", "query", "latency", "results"])
    
    def log_query(self, query, results, latency):
        """Log a query with its results and latency."""
        new_entry = pd.DataFrame({
            "timestamp": [datetime.now()],
            "query": [query],
            "latency": [latency],
            "results": [len(results)]
        })
        
        # Concatenate while preserving column types
        self.queries = pd.concat(
            [self.queries, new_entry],
            ignore_index=True,
            axis=0
        )
    
    def generate_report(self):
        """Generate a daily report of query analytics."""
        if self.queries.empty:
            return pd.DataFrame()  # Return empty DataFrame if no queries logged
        
        # Group by day and aggregate metrics
        report = self.queries.groupby(pd.Grouper(key="timestamp", freq="D")).agg({
            "query": "count",
            "latency": "mean",
            "results": "mean"
        }).rename(columns={
            "query": "query_count",
            "latency": "avg_latency",
            "results": "avg_results"
        })
        
        return report

# Usage Example
monitor = QueryMonitor()

# Simulate a query
start = time.time()
results = hybrid_search("Tech company risks")  # Replace with your hybrid search function
latency = time.time() - start

# Log the query
monitor.log_query("Tech company risks", results, latency)

# Generate and display the report
report = monitor.generate_report()
print(report)

C:\Users\Armghan\AppData\Local\Temp\ipykernel_15192\4007184976.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.queries = pd.concat(


            query_count  avg_latency avg_results
timestamp                                       
2025-03-01            1     0.308273         7.0


In [16]:
# =============================================
# Final Working VersionedIndex Class
# =============================================
import uuid
from datetime import datetime

class VersionedIndex:
    def __init__(self):
        self.versions = {}
    
    def commit(self, index, description):
        version_id = str(uuid.uuid4())
        index.storage_context.persist(persist_dir=f"./index_{version_id}")
        
        # Clean metadata for all documents
        cleaned_docs = []
        for doc in index.docstore.docs.values():
            # Convert all metadata values to strings
            safe_metadata = {
                k: v.isoformat() if isinstance(v, (date, datetime)) else str(v)
                for k, v in doc.metadata.items()
            }
            cleaned_docs.append({
                "id": doc.id_,
                "metadata": safe_metadata
            })
        
        # Create version entry with safe data
        self.versions[version_id] = {
            "timestamp": datetime.now().isoformat(),
            "description": description,
            "stats": {
                "doc_count": len(index.docstore.docs),
                "companies": list(set(
                    d["metadata"]["symbol"] for d in cleaned_docs
                    if "symbol" in d["metadata"]
                ))
            }
        }
        
        # Save with custom encoder
        with open("versions.json", "w") as f:
            json.dump(
                self.versions,
                f,
                cls=CustomJSONEncoder,
                indent=2,
                ensure_ascii=False
            )
            
        return version_id

In [17]:
# Simple search
results = hybrid_search("Companies with revenue over $100B")
print("Top Results:")
for idx, result in enumerate(results[:3]):
    # Extract revenue safely
    revenue_line = result.node.text.split('Revenue: ')[1]
    revenue = revenue_line.split('\n')[0]
    
    print(f"{idx+1}. {result.node.metadata['company']}")
    print(f"   Revenue: {revenue}")
    print(f"   Score: {result.score:.2f}\n")

Top Results:
1. Sutton, Curtis and Robertson
   Revenue: $274.80B
   Score: 0.49

2. Mullins, Yoder and Thomas
   Revenue: $325.77B
   Score: 0.48

3. Lopez, Wolfe and Hansen
   Revenue: $35.81B
   Score: 0.48



In [37]:
from llama_index.core import PromptTemplate

# Enhanced Analysis Prompt
EXPERT_ANALYSIS_PROMPT = PromptTemplate("""\
**Role**: Senior Financial Analyst at Goldman Sachs  
**Task**: Analyze SEC filings for sector comparison.  
**Context**: {context_str}  
**Query**: {query_str}  

**Response Format**:  
1. **Executive Summary**  
   - Summarize key findings in 2-3 sentences.  
   - Highlight major differences between sectors.  

2. **Sector Performance Overview**  
   - Create a table comparing:  
     - Revenue growth (YoY)  
     - Profit margins  
     - R&D spend (% of revenue)  
     - Cash reserves  
     - Debt ratios  
   - Include specific numbers from filings.  

3. **Risk Analysis**  
   - For each sector, list top 3 risks with:  
     - Risk name  
     - Severity (High/Medium/Low)  
     - Mitigation strategies cited in filings  
   - Use direct quotes where available.  

4. **Strategic Recommendations**  
   - Provide 3 actionable recommendations per sector.  
   - Base recommendations on filing insights.  

5. **Critical Insights from Filings**  
   - Include 2-3 direct quotes per sector.  
   - Explain the significance of each quote.  

6. **Emerging Trends**  
   - Identify 2-3 trends supported by filing data.  
   - Highlight potential future impacts.  

**Tone**: Professional, data-driven, and concise.  
**Style**: Use bullet points, tables, and clear headings.  
**Data**: Always cite specific filings and metrics.  
""")


def analyze_financials(results, query):
    """Generate expert analysis using actual document context"""
    # Build context from search results
    context = "\n\n".join([
        f"Document {i+1}: {result.node.metadata['company']} ({result.node.metadata['symbol']})\n"
        f"Filing Type: {result.node.metadata['filing_type']}\n"
        f"Content: {result.node.text[:1000]}..."
        for i, result in enumerate(results[:5])  # Use top 5 results
    ])
    
    # Generate analysis
    response = Settings.llm.complete(
        EXPERT_ANALYSIS_PROMPT.format(
            context_str=context,
            query_str=query
        )
    )
    return str(response)

In [19]:
def end_to_end_test(query):
    # Step 1: Hybrid Search
    start_time = time.time()
    results = hybrid_search(query)
    retrieval_time = time.time() - start_time
    
    # Step 2: LLM Analysis
    analysis = analyze_financials(query)
    
    # Step 3: Logging
    monitor.log_query(query, results, retrieval_time)
    
    # Step 4: Versioning
    if "critical" in query.lower():
        version_id = version_system.commit(index, f"Critical query: {query}")
    
    return {
        "analysis": analysis,
        "retrieval_time": f"{retrieval_time:.2f}s",
        "documents_used": len(results)
    }

# Test Complex Query
response = end_to_end_test(
    "Identify companies with decreasing gross margins but increasing R&D spend. "
    "What does this suggest about their strategic priorities?"
)
print(json.dumps(response, indent=2))

{
  "analysis": "1. Key Findings:\n- Company A has decreasing gross margins but increasing R&D spend.\n- Company B has decreasing gross margins but increasing R&D spend.\n- This suggests that both companies are prioritizing long-term strategic growth through investment in research and development, despite short-term decreases in profitability.\n\n2. Supporting Data:\n\nCompany A:\n- Gross Margin: 42.3% (2020) vs 39.5% (2021)\n- R&D Spend: $12.5B (2020) vs $15.3B (2021)\n\nCompany B:\n- Gross Margin: 38.7% (2020) vs 35.2% (2021)\n- R&D Spend: $8.9B (2020) vs $10.7B (2021)\n\n3. Recommendations:\n- Investors should be prepared for short-term decreases in profitability as these companies prioritize long-term growth.\n- However, the increasing R&D spend indicates a commitment to innovation and competitive positioning, which could lead to future revenue growth and market share expansion.\n- It is important to monitor the outcomes of these R&D investments and evaluate whether they are genera

In [20]:
version_system = VersionedIndex()
version_id = version_system.commit(index, "Final Production Version")

with open("versions.json") as f:
    print(json.dumps(json.load(f), indent=2))

{
  "2f537f77-3f11-40e7-8710-ff399a4b014d": {
    "timestamp": "2025-03-01T22:30:41.246878",
    "description": "Final Production Version",
    "stats": {
      "doc_count": 50,
      "companies": [
        "DRTD",
        "SZII",
        "NUCM",
        "DLTB",
        "YASV",
        "ZYSK",
        "LULT",
        "YNLW",
        "IJLS",
        "UREG",
        "TPPR",
        "VYOJ",
        "OWMP",
        "NTYY",
        "UKJX",
        "YCLE",
        "OWKV",
        "BIMB",
        "DUKZ",
        "CKWP",
        "HLYU",
        "PBZL",
        "UHYY",
        "BRXF",
        "EMDV",
        "HXRE",
        "NMSE",
        "OXBD",
        "PMQD",
        "XEWD",
        "OIEV",
        "XCIV",
        "NDFO",
        "KTJX",
        "EGFT",
        "AILB",
        "WUAO",
        "BVDZ",
        "LTCW",
        "DVWE",
        "OIJU",
        "LKQC",
        "UXMC",
        "FQVZ",
        "VFAS",
        "FBPE",
        "JOOS",
        "MVKO",
        "OHIP",
        "XUKS"
  

In [21]:
# Test empty results handling
response = end_to_end_test("Find companies in Mars colony sector")
print(response["analysis"])  # Should handle gracefully

1. Key Findings:
- There are three companies operating in the Mars colony sector with available SEC filings.
- The companies have varying financial performance, with revenues ranging from $60.54B to $331.87B and net income ranging from $6.06B to $47.25B.
- The companies have different employee counts, ranging from 82,736 to 431,139.
- The companies face various risk factors, including resource management, technology development, and regulatory challenges.

2. Supporting Data:

Company A:
- Revenue: $331.87B
- Net Income: $47.25B
- EPS: $1.99
- Employees: 431,139

Company B:
- Revenue: $124.21B
- Net Income: $13.17B
- EPS: $19.62
- Employees: 331,217

Company C:
- Revenue: $60.54B
- Net Income: $20.34B
- EPS: $1.51
- Employees: 82,736

3. Recommendations:
- Investors should consider the financial performance and risk factors of each company before making investment decisions.
- Companies in the Mars colony sector should focus on managing resources, developing technology, and navigating 

In [39]:
class FinancialQuerySystem:
    def __init__(self, index, monitor):
        self.index = index
        self.monitor = monitor

    def execute_query(self, query):
        """Execute a financial query and return expert analysis."""
        if not query.strip():
            return "Error: Query cannot be empty."

        print(f"\nProcessing query: '{query}'")
        
        try:
            # Step 1: Hybrid Search
            start_time = time.time()
            results = hybrid_search(query)
            latency = time.time() - start_time
            
            # Log the query
            self.monitor.log_query(query, results, latency)
            
            # Step 2: Generate Expert Analysis
            analysis = analyze_financials(results, query)
            
            return analysis

        except Exception as e:
            return f"Error processing query: {str(e)}"


# Initialize the system
query_system = FinancialQuerySystem(index, monitor)

# Interactive query loop
if __name__ == "__main__":
    print("=== Financial Query System ===")
    print("Enter your financial query (or type 'exit' to quit).")
    
    while True:
        query = input("\nYour query: ").strip()
        if query.lower() == "exit":
            print("Exiting system. Goodbye!")
            break
        
        response = query_system.execute_query(query)
        print("\n" + response)

=== Financial Query System ===
Enter your financial query (or type 'exit' to quit).

Processing query: 'what are the top risks for tech companies in 2023?'

**Executive Summary**

After analyzing SEC filings for five tech companies across various sectors, we identified key trends and risks. While revenue growth and R&D spending remain strong, profit margins and cash reserves vary. Debt ratios are generally low, but some companies face significant legal and regulatory risks.

**Sector Performance Overview**

| Sector/Company | Revenue Growth (YoY) | Profit Margins | R&D Spend (% of Revenue) | Cash Reserves | Debt Ratios |
| --- | --- | --- | --- | --- | --- |
| Carter-Miller (NUCM) | 12% | 15% | 20% | $3B | 0.25 |
| James-Esparza (QHZT) | 8% | 18% | 15% | $2B | 0.30 |
| Williams Group (JOOS) | 15% | 12% | 22% | $4B | 0.15 |
| Chan Ltd (ZTKM) | 10% | 14% | 18% | $2.5B | 0.20 |
| Trujillo and Sons (UREG) | 9% | 13% | 17% | $1.5B | 0.22 |

**Risk Analysis**

*Carter-Miller (NUCM)*
1. Regul